In [ ]:
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "meal-planner-demo"

print("✅ Environment loaded")


In [ ]:
from main_agent import create_meal_planning_agent, MealPlannerState
from nutrition_optimizer import MealPlan
from food_database import get_food_database
from langchain_core.messages import HumanMessage
from IPython.display import display, Markdown
import json

print("✅ Imports successful")


In [ ]:
# Create the agent
agent = create_meal_planning_agent()

# Initialize state function
def create_initial_state():
    return {
        "messages": [],
        "current_meal_plan": MealPlan(),
        "user_profile": {},
        "food_database": get_food_database(),
        "conversation_phase": "gathering_info",
        "optimization_history": []
    }

# Helper function for conversation
def chat_with_agent(message: str, state: dict, config: dict, show_full_response=False):
    print(f"👤 **User:** {message}\n")
    
    # Add user message
    state["messages"].append(HumanMessage(content=message))
    
    # Get agent response
    response = agent.invoke(state, config)
    agent_message = response["messages"][-1].content
    
    if show_full_response:
        print(f"🤖 **Agent:** {agent_message}\n")
    else:
        # Show truncated version
        preview = agent_message[:300] + "..." if len(agent_message) > 300 else agent_message
        print(f"🤖 **Agent:** {preview}\n")
    
    print("-" * 80)
    return response

print("✅ Agent initialized")


In [ ]:
# Start a new conversation
state = create_initial_state()
config = {"configurable": {"thread_id": "vegetarian_demo"}}

print("🎯 **Scenario: Vegetarian High-Protein Meal Planning**\n")

# Turn 1: Introduction and requirements
state = chat_with_agent(
    "Hi! I'm a vegetarian trying to build muscle. I need about 2200 calories per day with high protein. Can you help me create a meal plan?",
    state, config
)


In [ ]:
# Turn 2: Set nutrition goals
state = chat_with_agent(
    "Yes, please set my nutrition goals for 2200 calories with a high-protein vegetarian focus.",
    state, config, show_full_response=True
)


In [ ]:
# Turn 3: Request breakfast suggestions
state = chat_with_agent(
    "Great! Now can you suggest some high-protein vegetarian breakfast options?",
    state, config, show_full_response=True
)


In [ ]:
from nutrition_optimizer import (
    NutritionOptimizer, ConstraintSet, NutrientConstraint,
    Meal, FoodItem
)

# Create a sample meal to optimize
sample_meal = Meal(
    id="breakfast1",
    name="Morning Protein Bowl",
    meal_type="breakfast",
    foods={
        "greek_yogurt": 1,
        "berries": 0.5,
        "almonds": 0.25
    }
)

# Define nutrition goals for breakfast
breakfast_profile = ConstraintSet(
    calories=NutrientConstraint(minimum=600, target=733, maximum=800),
    protein=NutrientConstraint(minimum=35, target=40, maximum=45),
    carbohydrates=NutrientConstraint(minimum=70, target=85, maximum=100),
    fat=NutrientConstraint(minimum=20, target=25, maximum=30)
)

# Create optimizer
food_db = get_food_database()
optimizer = NutritionOptimizer(food_db, breakfast_profile)

# Show initial nutrition
initial_nutrition = sample_meal.calculate_nutrition(food_db)
print("📊 **Initial Meal Nutrition:**")
for nutrient, value in initial_nutrition.items():
    target = getattr(breakfast_profile, nutrient).target
    print(f"- {nutrient.capitalize()}: {value:.1f}g (target: {target:.1f}g)")

initial_fitness = optimizer.calculate_fitness(MealPlan(breakfast=sample_meal))
print(f"\n🎯 **Initial Fitness Score:** {initial_fitness:.2f} (lower is better)")


In [ ]:
# Optimize the meal
print("🔄 **Optimizing meal...** (this may take a moment)\n")

optimized_meal, fitness = optimizer.hill_climb_meal(sample_meal, max_iterations=100)

# Show optimized nutrition
optimized_nutrition = optimized_meal.calculate_nutrition(food_db)
print("📊 **Optimized Meal Nutrition:**")
for nutrient, value in optimized_nutrition.items():
    target = getattr(breakfast_profile, nutrient).target
    improvement = abs(value - target) - abs(initial_nutrition[nutrient] - target)
    status = "✅ Better" if improvement < 0 else "➡️ Similar" if abs(improvement) < 1 else "❌ Worse"
    print(f"- {nutrient.capitalize()}: {value:.1f}g (target: {target:.1f}g) {status}")

print(f"\n🎯 **Optimized Fitness Score:** {fitness:.2f}")
print(f"📈 **Improvement:** {initial_fitness - fitness:.2f}")

# Show changes
print("\n🔧 **Recommended Changes:**")
suggestions = optimizer.suggest_meal_improvement(sample_meal, "breakfast")
if suggestions["changes"]:
    for change in suggestions["changes"]:
        print(f"- {change}")
else:
    print("- No significant changes recommended")
